In [ ]:
import random

# Problem size
N = 8  # 8 queens

# EA settings 
POP_SIZE = 60
TOURNAMENT_K = 5
MAX_EVALS = 10_000

# Change this to 0.60, 0.80, 1.00 for your three required runs
MUTATION_PROB = 01.0

# How much to print while running
VERBOSE = True


# --------- Utilities ---------
def make_individual():
    """
    An individual is a permutation of [1..8].
    Index = column (0..7), value = row (1..8).
    This guarantees no row and no column clashes.
    """
    p = list(range(1, N + 1))
    random.shuffle(p)
    return p


def fitness(perm):
    """
    Number of *attacking* pairs on diagonals.
    0 means a valid 8-queens solution.
    """
    attacks = 0
    for c1 in range(N):
        r1 = perm[c1]
        for c2 in range(c1 + 1, N):
            r2 = perm[c2]
            # Same diagonal if row diff == column diff
            if abs(r1 - r2) == abs(c1 - c2):
                attacks += 1
    return attacks


def select_parent(pop_with_fit):
    """
    Tournament selection with k=5.
    Pick 5 at random, return the best one's genome.
    pop_with_fit is a list of (genome, fit).
    """
    candidates = random.sample(pop_with_fit, TOURNAMENT_K)
    candidates.sort(key=lambda x: x[1])  # lower fitness is better
    # return a copy of the winner genome
    return candidates[0][0][:]


def cut_and_crossfill(p1, p2):
    """
    One-cut 'cut-and-crossfill' crossover for permutations.
    Child1 = prefix from p1 + remaining numbers in the order they appear in p2.
    Child2 is symmetric.
    """
    cut = random.randint(1, N - 1)

    prefix1 = p1[:cut]
    prefix2 = p2[:cut]

    child1 = prefix1 + [x for x in p2 if x not in prefix1]
    child2 = prefix2 + [x for x in p1 if x not in prefix2]
    return child1, child2


def mutate_swap(perm):
    """Swap two random positions in-place."""
    i, j = random.sample(range(N), 2)
    perm[i], perm[j] = perm[j], perm[i]


def print_board(perm):
    """
    print the board for a permutation.
    perm[col] = row (1..8). 'Q' marks the queen, '.' is empty.
    """
    for row in range(1, N + 1):
        line = []
        for col in range(N):
            if perm[col] == row:
                line.append("Q")
            else:
                line.append(".")
        print(" ".join(line))
    print()


# --------- Main EA loop ---------
def run_ea(seed=None):
    if seed is not None:
        random.seed(seed)

    # 1) Start with a random population and evaluate it
    population = [make_individual() for _ in range(POP_SIZE)]
    pop_with_fit = [(ind, fitness(ind)) for ind in population]
    evaluations = POP_SIZE

    best_over_time = []
    avg_over_time = []

    generation = 0
    while evaluations < MAX_EVALS:
        # Keep the population sorted so best individual is at index 0
        pop_with_fit.sort(key=lambda x: x[1])
        best_fit = pop_with_fit[0][1]
        avg_fit = sum(f for _, f in pop_with_fit) / len(pop_with_fit)
        best_over_time.append(best_fit)
        avg_over_time.append(avg_fit)

        if VERBOSE and (generation < 10 or generation % 50 == 0):
            print(f"gen {generation:4} | best={best_fit:2d} | avg={avg_fit:5.2f}")

        # Stop if we found a solution (fitness 0)
        if best_fit == 0:
            break

        # 2) Select two parents (tournament-5)
        p1 = select_parent(pop_with_fit)
        p2 = select_parent(pop_with_fit)

        # 3) Crossover to make two children
        c1, c2 = cut_and_crossfill(p1, p2)

        # 4) Mutate each child with probability MUTATION_PROB
        if random.random() < MUTATION_PROB:
            mutate_swap(c1)
        if random.random() < MUTATION_PROB:
            mutate_swap(c2)

        # 5) Evaluate children
        off1 = (c1, fitness(c1))
        off2 = (c2, fitness(c2))
        evaluations += 2

        # 6) Survivor selection: (μ + λ) — add kids, then keep best POP_SIZE
        pop_with_fit.extend([off1, off2])
        pop_with_fit.sort(key=lambda x: x[1])
        pop_with_fit = pop_with_fit[:POP_SIZE]

        generation += 1

    # Final stats
    pop_with_fit.sort(key=lambda x: x[1])
    best = pop_with_fit[0][0]
    best_fit = pop_with_fit[0][1]

    print("\n--- Finished ---")
    print(f"Evaluations used: {evaluations}")
    print(f"Best fitness: {best_fit}")
    print(f"Best individual (perm): {best}\n")

    if best_fit == 0:
        print("A valid 8-queens solution:\n")
        print_board(best)
    else:
        print("No perfect solution found. Best board so far:\n")
        print_board(best)

    return {
        "best": best,
        "best_fitness": best_fit,
        "evaluations": evaluations,
        "best_over_time": best_over_time,
        "avg_over_time": avg_over_time,
    }


# --------- Run it ---------
if __name__ == "__main__":
    # run this 3 times with MUTATION_PROB = 0.60, 0.80, 1.00
    # (change the number above, re-run, and record the results)
    run_ea(seed=42)  # set or remove seed if you want different randomness
